# 구글 드라이브 연결 + 필요한 라이브러리 설치

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
base_dir = "/gdrive/MyDrive/SSAFY-필드프로젝트/"

In [ ]:
!pip install pymysql

In [ ]:
!pip install fasttext

In [ ]:
!pip install konlpy
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

# 라이브러리 IMPORT

In [4]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기

In [5]:
# 전처리된 책 데이터 불러오기
books = pd.read_pickle(base_dir + "data/book_data.pkl")

# 책 데이터 임베딩 처리한 데이터 불러오기
book_embedding_list = np.load(base_dir + "data/embedding_matrix.npy")

In [6]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88416 entries, 0 to 88415
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   book_isbn            88416 non-null  int64 
 1   book_title           88416 non-null  object
 2   book_author          88416 non-null  object
 3   book_publisher       88416 non-null  object
 4   book_contents        88416 non-null  object
 5   book_description     88416 non-null  object
 6   cleaned_title        88416 non-null  object
 7   cleaned_contents     88416 non-null  object
 8   cleaned_description  88416 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.1+ MB


In [7]:
len(book_embedding_list)

88416

# 추천 알고리즘

In [8]:
def recommendations(isbn):
    recommended_books = books[['book_isbn','book_title', 'book_author']]

    # 책의 ISBN을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(books.index, index = books['book_isbn']).drop_duplicates()    
    idx = indices[isbn]

    # 해당 책과 다른 책들의 cosine similarity 계산
    cos_similarity = cosine_similarity([book_embedding_list[idx]], book_embedding_list)

    # 입력된 책과 제목이 유사한 책 5개 선정.
    sim_scores = list(enumerate(cos_similarity[0]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = recommended_books.iloc[book_indices].reset_index(drop=True)

    return recommend

In [9]:
recommend = recommendations(9788930106924)
recommend

[(8026, 0.98234844), (26450, 0.9823119), (42719, 0.9821646), (12555, 0.98156804), (18699, 0.9807234)]


,book_isbn,book_title,book_author
0,9788932022505,가나,정용준 지음
1,9788961706889,뼈들이 노래한다,"숀 탠 지음, 황윤영 옮김"
2,9788986270433,즉흥연기,"키스 존스톤 지음, 이민아 옮김"
3,9788937484025,우스운 사랑들,"밀란 쿤데라 지음, 방미경 옮김"
4,9788954656696,무엇이든 가능하다,"엘리자베스 스트라우트 지음, 정연희 옮김"
